In [0]:
%pip install snowflake-snowpark-python pandas
dbutils.library.restartPython()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 61.0 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Not uninstalling cffi at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-41ef3f11-dd68-4869-82ff-1b2541c8dbbd
    Can't uninstall 'cffi'. No files were found to uninstall.
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Not uninstalling cryptography at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-41ef3f11-dd68-4869-82ff-1b2541c8dbbd
    Can't uninstall 'cryptography'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use up

In [0]:
%pip install snowflake-snowpark-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/176.1 kB ? eta -:--:--
     ━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/176.1 kB 1.3 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━ 71.7/176.1 kB 916.2 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━ 102.4/176.1 kB 935.4 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/77.0 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.8 MB ? eta -:--:--
   ━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2/1.8 MB 8.9 MB/s eta 0:00:01
   ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.4/1.8 MB 6.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━ 0.7/1.8 MB 6.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━ 1.4/1.8 MB 10.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.8/1.8 MB 11.3 MB/s eta 0:00:01


In [0]:
%restart_python

Connect Snowflake Session

In [0]:
from snowflake.snowpark import Session

connection_parameters = {
    "account": "DWGAELR-KQ25919",
    "user": "RAGAVIVENUGOPAL",
    "password": "Ragavi@123456789",
    "role": "ACCOUNTADMIN",
    "warehouse": "ITG_WAREHOUSE",
    "database": "ITG_SALES_DB",
    "schema": "RAW"
}
session = Session.builder.configs(connection_parameters).create()


Create File Format & Stage

In [0]:

session.sql("""
CREATE OR REPLACE FILE FORMAT csv_format
  TYPE = 'CSV'
  FIELD_DELIMITER = ','
  SKIP_HEADER = 1
  NULL_IF = ('NULL', 'null')
  EMPTY_FIELD_AS_NULL = TRUE;
""").collect()

session.sql("""
CREATE OR REPLACE STAGE azure_stage
  URL = 'azure://itgretailstorage.blob.core.windows.net/monthly-sales/'
  CREDENTIALS = (AZURE_SAS_TOKEN = '?sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-10-24T13:14:20Z&st=2025-10-24T04:59:20Z&spr=https&sig=9R6uMAylOZ5POtq6QdwkrYBx6y4Mmr9c36OQUs9A3xU%3D')
  FILE_FORMAT = csv_format;
""").collect()

print("File format and stage created successfully")

File format and stage created successfully


In [0]:
# Test stage connection
try:
    result = session.sql("LIST @azure_stage").collect()
    print("Stage connection successful! Files found:")
    for row in result:
        print(f" - {row['name']}")
except Exception as e:
    print(f"Error listing files: {str(e)}")

Stage connection successful! Files found:
 - azure://itgretailstorage.blob.core.windows.net/monthly-sales/Retail_Sales__500_rows__Preview.csv


In [0]:
# Create raw table schema matching your CSV
session.sql("""
CREATE OR REPLACE TABLE raw_sales_data (
    OrderID STRING,
    OrderDate DATE,
    MonthOfSale STRING,
    CustomerID STRING,
    CustomerName STRING,
    Country STRING,
    Region STRING,
    City STRING,
    Category STRING,
    Subcategory STRING,
    Quantity INTEGER,
    Discount NUMBER(10,2),
    Sales NUMBER(10,2),
    Profit NUMBER(10,2),
    FileName STRING,
    LoadTimestamp TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP()
);
""").collect()

print("Raw sales table created successfully")

Raw sales table created successfully


In [0]:
# Ingest data from Azure to Snowflake
session.sql("""
COPY INTO raw_sales_data (
  OrderID, OrderDate, MonthOfSale, CustomerID, CustomerName,
  Country, Region, City, Category, Subcategory,
  Quantity, Discount, Sales, Profit, FileName
)
FROM (
  SELECT $1, $2, $3, $4, $5, $6, $7, $8, $9, $10,
         $11, $12, $13, $14, METADATA$FILENAME
  FROM @azure_stage/Retail_Sales__500_rows__Preview.csv
)
FILE_FORMAT = (FORMAT_NAME = csv_format)
ON_ERROR = 'CONTINUE';
""").collect()


[Row(file='azure://itgretailstorage.blob.core.windows.net/monthly-sales/Retail_Sales__500_rows__Preview.csv', status='LOADED', rows_parsed=25, rows_loaded=25, error_limit=25, errors_seen=0, first_error=None, first_error_line=None, first_error_character=None, first_error_column_name=None)]

Load Data

In [0]:
session.sql("""
COPY INTO raw_sales_data (
  OrderID, OrderDate, MonthOfSale, CustomerID, CustomerName,
  Country, Region, City, Category, Subcategory,
  Quantity, Discount, Sales, Profit, FileName
)
FROM (
  SELECT $1, $2, $3, $4, $5, $6, $7, $8, $9, $10,
         $11, $12, $13, $14, METADATA$FILENAME
  FROM @azure_stage/sales_october.csv
)
FILE_FORMAT = (FORMAT_NAME = csv_format)
ON_ERROR = 'CONTINUE';
""").collect()

[Row(status='Copy executed with 0 files processed.')]

In [0]:
# Verify raw data
result = session.sql("SELECT COUNT(*) as total_rows FROM raw_sales_data").collect()
print(f"Total rows in raw table: {result[0]['TOTAL_ROWS']}")

print("Sample raw data:")
session.sql("SELECT * FROM raw_sales_data LIMIT 5").show()

Total rows in raw table: 25
Sample raw data:
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDERID"     |"ORDERDATE"  |"MONTHOFSALE"  |"CUSTOMERID"  |"CUSTOMERNAME"  |"COUNTRY"  |"REGION"  |"CITY"   |"CATEGORY"       |"SUBCATEGORY"  |"QUANTITY"  |"DISCOUNT"  |"SALES"   |"PROFIT"  |"FILENAME"                           |"LOADTIMESTAMP"             |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|ORD-5F8D6F0C  |2024-10-08   |2024-10        |CUST1000      |Ananya Sharma   |India      |South     |Mumbai   |Office Supplies  |Paper          |9           |0.00        |2

In [0]:
# Switch to clean schema and create clean data table
session.sql("USE SCHEMA ITG_SALES_DB").collect()

session.sql("""
CREATE OR REPLACE TABLE clean_sales_data AS
SELECT 
    OrderID,
    OrderDate,
    MonthOfSale,
    CustomerID,
    CustomerName,
    Country,
    Region,
    City,
    Category,
    Subcategory,
    Quantity,
    Discount,
    Sales,
    Profit,
    -- Data validation and calculations
    CASE WHEN Sales != Quantity * (Sales/NULLIF(Quantity,0)) THEN Sales ELSE Sales END AS ValidatedSales,
    -- Date parts for analysis
    YEAR(OrderDate) AS OrderYear,
    MONTH(OrderDate) AS OrderMonth,
    QUARTER(OrderDate) AS OrderQuarter,
    -- Business metrics
    Sales * Discount AS DiscountAmount,
    Profit / NULLIF(Sales, 0) AS ProfitMargin,
    LoadTimestamp
FROM ITG_SALES_DB.RAW.raw_sales_data
WHERE OrderDate IS NOT NULL AND Sales > 0;
""").collect()

print("Clean sales data table created")

Clean sales data table created


In [0]:
# Create aggregated views for Power BI
session.sql("""
CREATE OR REPLACE VIEW sales_summary_monthly AS
SELECT
    Region,
    Category,
    OrderYear,
    OrderMonth,
    COUNT(*) AS TotalOrders,
    SUM(Sales) AS TotalSales,
    SUM(Profit) AS TotalProfit,
    AVG(Sales) AS AvgOrderValue,
    SUM(Quantity) AS TotalQuantity,
    COUNT(DISTINCT CustomerID) AS UniqueCustomers
FROM clean_sales_data
GROUP BY Region, Category, OrderYear, OrderMonth
ORDER BY OrderYear, OrderMonth, Region;
""").collect()

print("Aggregated views created successfully")

Aggregated views created successfully


In [0]:
# Create Power BI optimized view
session.sql("""
CREATE OR REPLACE VIEW vw_powerbi_sales_dashboard AS
SELECT 
    cs.*,
    sm.TotalSales AS RegionMonthlySales,
    sm.TotalProfit AS RegionMonthlyProfit,
    sm.UniqueCustomers AS RegionMonthlyCustomers
FROM clean_sales_data cs
LEFT JOIN sales_summary_monthly sm 
    ON cs.Region = sm.Region 
    AND cs.Category = sm.Category
    AND cs.OrderYear = sm.OrderYear 
    AND cs.OrderMonth = sm.OrderMonth;
""").collect()

print("Power BI view created successfully")

Power BI view created successfully


In [0]:
# Data quality checks
checks = [
    "SELECT COUNT(*) AS total_rows FROM raw_sales_data",
    "SELECT COUNT(*) AS clean_rows FROM clean_sales_data", 
    "SELECT COUNT(DISTINCT Region) AS region_count FROM clean_sales_data",
    "SELECT MIN(OrderDate) AS earliest_date, MAX(OrderDate) AS latest_date FROM clean_sales_data",
    "SELECT SUM(Sales) AS total_sales, SUM(Profit) AS total_profit FROM clean_sales_data"
]

print("Data Quality Check Results:")
for check in checks:
    try:
        result = session.sql(check).collect()
        print(f"{check}: {result[0]}")
    except Exception as e:
        print(f"Error in check {check}: {str(e)}")

Data Quality Check Results:
Error in check SELECT COUNT(*) AS total_rows FROM raw_sales_data: (1304): 01bfeaea-0001-672c-000c-603a00039196: 002003 (42S02): SQL compilation error:
Object 'RAW_SALES_DATA' does not exist or not authorized.
SELECT COUNT(*) AS clean_rows FROM clean_sales_data: Row(CLEAN_ROWS=25)
SELECT COUNT(DISTINCT Region) AS region_count FROM clean_sales_data: Row(REGION_COUNT=5)
SELECT MIN(OrderDate) AS earliest_date, MAX(OrderDate) AS latest_date FROM clean_sales_data: Row(EARLIEST_DATE=datetime.date(2024, 2, 27), LATEST_DATE=datetime.date(2025, 9, 15))
SELECT SUM(Sales) AS total_sales, SUM(Profit) AS total_profit FROM clean_sales_data: Row(TOTAL_SALES=Decimal('1066026.50'), TOTAL_PROFIT=Decimal('180647.47'))


In [0]:
# Final verification and sample data
print("Final verification - Sample from Power BI view:")
session.sql("SELECT * FROM vw_powerbi_sales_dashboard LIMIT 5").show()

print("Summary statistics:")
session.sql("""
SELECT 
    COUNT(*) AS total_records,
    COUNT(DISTINCT CustomerID) AS unique_customers,
    COUNT(DISTINCT Region) AS regions_covered,
    SUM(Sales) AS total_sales,
    SUM(Profit) AS total_profit,
    AVG(ProfitMargin) AS avg_profit_margin
FROM vw_powerbi_sales_dashboard
""").show()

Final verification - Sample from Power BI view:
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDERID"     |"ORDERDATE"  |"MONTHOFSALE"  |"CUSTOMERID"  |"CUSTOMERNAME"  |"COUNTRY"  |"REGION"  |"CITY"   |"CATEGORY"       |"SUBCATEGORY"  |"QUANTITY"  |"DISCOUNT"  |"SALES"   |"PROFIT"  |"VALIDATEDSALES"  |"ORDERYEAR"  |"ORDERMONTH"  |"ORDERQUARTER"  |"DISCOUNTAMOUNT"  |"PROFITMARGIN"  |"LOADTIMESTAMP"             |"REGIONMONTHLYSALES"  |"REGIONMONTHLYPROFIT"  |"REGIONMONTHLYCUSTOMERS"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# Close session
session.close()
print("Snowpark session closed")
print("Pipeline execution completed successfully!")
print("\nNext steps: Connect Power BI to Snowflake using:")
print("Database: ITTG_SALES_DB")
print("Schema: CLEAN_DATA") 
print("View: VW_POWERBI_SALES_DASHBOARD")

Snowpark session closed
Pipeline execution completed successfully!

Next steps: Connect Power BI to Snowflake using:
Database: ITTG_SALES_DB
Schema: CLEAN_DATA
View: VW_POWERBI_SALES_DASHBOARD
